In [ ]:
%cd ..

/home/jupyter/Azure_repo/template-mlops


# MAIN PROCESS

In [ ]:
#==== Importing necessary libraries ====#
import pandas as pd
from sklearn.datasets import load_iris
from src.utils import preprocess_data, train_model, evaluate_model, get_metrics

#==== Loading the Iris dataset ====#
iris = load_iris()
data = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
data['target'] = iris['target']
# Define a mapping dictionary for the columns
column_mapping = {
    'sepal length (cm)': 'sepal_length_cm',
    'sepal width (cm)': 'sepal_width_cm',
    'petal length (cm)': 'petal_length_cm',
    'petal width (cm)': 'petal_width_cm',
}

# Rename the columns using the mapping
data.rename(columns=column_mapping, inplace=True)

#==== Preprocessing the data ====#
train_df, test_df, scaler = preprocess_data(data)

#==== Training the model ====#
# Separating features and target for training
X_train = train_df.drop(columns=['target'])
y_train = train_df['target']

model = train_model(X_train, y_train)

#==== Evaluating the model ====#
X_test = test_df.drop(columns=['target'])
y_test = test_df['target']
predictions = model.predict(X_test)

#==== Getting metrics ====#
logs, conf_matrix = get_metrics(y_test, predictions)
print("Evaluation Metrics:")
for key, value in logs.items():
    print(f"{key}: {value}")

Evaluation Metrics:
precision: 1.0
recall: 1.0
f1: 1.0


In [ ]:
columns = list(data.columns)

In [ ]:
columns[:-1]

['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm']

In [ ]:
columns[-1]

'target'

In [ ]:
target_classes = sorted(y_train.unique())

In [ ]:
type(target_classes)

list

In [ ]:
random_sample = data.sample(n=min(100, len(data)), random_state=42) # The random_state ensures reproducibility

In [ ]:
random_sample

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
73,6.1,2.8,4.7,1.2,1
18,5.7,3.8,1.7,0.3,0
118,7.7,2.6,6.9,2.3,2
78,6.0,2.9,4.5,1.5,1
76,6.8,2.8,4.8,1.4,1
...,...,...,...,...,...
89,5.5,2.5,4.0,1.3,1
8,4.4,2.9,1.4,0.2,0
13,4.3,3.0,1.1,0.1,0
119,6.0,2.2,5.0,1.5,2


In [ ]:
# Import the required libraries
from google.cloud import aiplatform

# Initialize the AI Platform SDK
aiplatform.init(project="pe-fcor-ec-coea-explore-dev", location="us-east4")

# Define display name for the endpoint
display_name = "iris_test"

# Create the endpoint
endpoint = aiplatform.Endpoint.create(display_name=display_name)

# Print the endpoint details
print("Endpoint created:", endpoint.resource_name)


Creating Endpoint
Create Endpoint backing LRO: projects/121305041613/locations/us-east4/endpoints/2183973140391002112/operations/6168231644521037824
Endpoint created. Resource name: projects/121305041613/locations/us-east4/endpoints/2183973140391002112
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/121305041613/locations/us-east4/endpoints/2183973140391002112')
Endpoint created: projects/121305041613/locations/us-east4/endpoints/2183973140391002112


In [ ]:
endpoint.name

'2183973140391002112'

In [ ]:
from typing import Dict, List, Union

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if isinstance(instances, list) else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

In [ ]:
instances = { "sepal length (cm)": "6.1", 
"sepal width (cm)": "2.8",
"petal length (cm)":"4.7",
"petal width (cm)":"1.2"}

In [ ]:
instances = {"instances": [6.1, 2.8, 4.7, 1.2]}

In [ ]:
predict_custom_trained_model_sample(
    project="121305041613",
    endpoint_id="2183973140391002112",
    location="us-east4",
    instances=instances
)

NotFound: 404 Endpoint `projects/121305041613/locations/us-east4/endpoints/2183973140391002112` not found.

In [ ]:
PROJECT_ID = 'pe-fcor-ec-coea-explore-dev'
REGION = 'us-east4'
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)

#https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model
model= aiplatform.Model(model_name='6233245767071432704')

In [ ]:
model.name

'6233245767071432704'

In [ ]:
from google.cloud import aiplatform
model_name = 'iris-model'
def get_endpoint_by_display_name(display_name: str):
    # Initialize the AI Platform client
    aiplatform.init()

    # List all available endpoints
    endpoints = aiplatform.Endpoint.list()

    # Filter the endpoints by the given display_name
    for endpoint in endpoints:
        if endpoint.display_name == display_name:
            return endpoint
    
    return None

# Example usage
display_name = model_name + "-endpoint"
endpoint = get_endpoint_by_display_name(display_name)

if endpoint:
    print(f"Endpoint with display name '{display_name}' found:", endpoint)
else:
    print(f"No endpoint found with display name '{display_name}'.")
    # You can create the endpoint here if it doesn't exist
    endpoint = aiplatform.Endpoint.create(display_name=display_name)


Endpoint with display name 'iris-model-endpoint' found: <google.cloud.aiplatform.models.Endpoint object at 0x7f5075de5990> 
resource name: projects/121305041613/locations/us-east4/endpoints/1940778760512995328


In [ ]:
%cd ..

/home/jupyter/Azure_repo/template-mlops


# MAIN PROCESS

In [ ]:
#==== Importing necessary libraries ====#
import pandas as pd
from sklearn.datasets import load_iris
from src.utils import preprocess_data, train_model, evaluate_model, get_metrics

#==== Loading the Iris dataset ====#
iris = load_iris()
data = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
data['target'] = iris['target']
# Define a mapping dictionary for the columns
column_mapping = {
    'sepal length (cm)': 'sepal_length_cm',
    'sepal width (cm)': 'sepal_width_cm',
    'petal length (cm)': 'petal_length_cm',
    'petal width (cm)': 'petal_width_cm',
}

# Rename the columns using the mapping
data.rename(columns=column_mapping, inplace=True)

#==== Preprocessing the data ====#
train_df, test_df, scaler = preprocess_data(data)

#==== Training the model ====#
# Separating features and target for training
X_train = train_df.drop(columns=['target'])
y_train = train_df['target']

model = train_model(X_train, y_train)

#==== Evaluating the model ====#
X_test = test_df.drop(columns=['target'])
y_test = test_df['target']
predictions = model.predict(X_test)

#==== Getting metrics ====#
logs, conf_matrix = get_metrics(y_test, predictions)
print("Evaluation Metrics:")
for key, value in logs.items():
    print(f"{key}: {value}")

Evaluation Metrics:
precision: 1.0
recall: 1.0
f1: 1.0


In [ ]:
data.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [ ]:
target_classes = sorted(y_train.unique())

In [ ]:
type(target_classes)

list

In [ ]:
random_sample = data.sample(n=min(100, len(data)), random_state=42) # The random_state ensures reproducibility

In [ ]:
random_sample

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
73,6.1,2.8,4.7,1.2,1
18,5.7,3.8,1.7,0.3,0
118,7.7,2.6,6.9,2.3,2
78,6.0,2.9,4.5,1.5,1
76,6.8,2.8,4.8,1.4,1
...,...,...,...,...,...
89,5.5,2.5,4.0,1.3,1
8,4.4,2.9,1.4,0.2,0
13,4.3,3.0,1.1,0.1,0
119,6.0,2.2,5.0,1.5,2


In [ ]:
# Import the required libraries
from google.cloud import aiplatform

# Initialize the AI Platform SDK
aiplatform.init(project="pe-fcor-ec-coea-explore-dev", location="us-east4")

# Define display name for the endpoint
display_name = "iris_test"

# Create the endpoint
endpoint = aiplatform.Endpoint.create(display_name=display_name)

# Print the endpoint details
print("Endpoint created:", endpoint.resource_name)


Creating Endpoint
Create Endpoint backing LRO: projects/121305041613/locations/us-east4/endpoints/2183973140391002112/operations/6168231644521037824
Endpoint created. Resource name: projects/121305041613/locations/us-east4/endpoints/2183973140391002112
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/121305041613/locations/us-east4/endpoints/2183973140391002112')
Endpoint created: projects/121305041613/locations/us-east4/endpoints/2183973140391002112


In [ ]:
endpoint.name

'2183973140391002112'

In [ ]:
from typing import Dict, List, Union

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if isinstance(instances, list) else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

In [ ]:
instances = { "sepal length (cm)": "6.1", 
"sepal width (cm)": "2.8",
"petal length (cm)":"4.7",
"petal width (cm)":"1.2"}

In [ ]:
instances = {"instances": [6.1, 2.8, 4.7, 1.2]}

In [ ]:
predict_custom_trained_model_sample(
    project="121305041613",
    endpoint_id="2183973140391002112",
    location="us-east4",
    instances=instances
)

NotFound: 404 Endpoint `projects/121305041613/locations/us-east4/endpoints/2183973140391002112` not found.

In [ ]:
PROJECT_ID = 'pe-fcor-ec-coea-explore-dev'
REGION = 'us-east4'
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)

#https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model
model= aiplatform.Model(model_name='6233245767071432704')

In [ ]:
model.name

'6233245767071432704'

In [ ]:
from google.cloud import aiplatform
model_name = 'iris-model'
def get_endpoint_by_display_name(display_name: str):
    # Initialize the AI Platform client
    aiplatform.init()

    # List all available endpoints
    endpoints = aiplatform.Endpoint.list()

    # Filter the endpoints by the given display_name
    for endpoint in endpoints:
        if endpoint.display_name == display_name:
            return endpoint
    
    return None

# Example usage
display_name = model_name + "-endpoint"
endpoint = get_endpoint_by_display_name(display_name)

if endpoint:
    print(f"Endpoint with display name '{display_name}' found:", endpoint)
else:
    print(f"No endpoint found with display name '{display_name}'.")
    # You can create the endpoint here if it doesn't exist
    endpoint = aiplatform.Endpoint.create(display_name=display_name)


Endpoint with display name 'iris-model-endpoint' found: <google.cloud.aiplatform.models.Endpoint object at 0x7f5075de5990> 
resource name: projects/121305041613/locations/us-east4/endpoints/1940778760512995328


In [ ]:
predict_custom_trained_model_sample(
    project="121305041613",
    endpoint_id="1940778760512995328",
    location="us-east4",
    instances=instances
)

NotFound: 404 Endpoint `projects/121305041613/locations/us-east4/endpoints/1940778760512995328` not found.

In [ ]:
instances = [
 [6.1, 2.8, 4.7, 1.2],
 [5.7, 3.8, 1.7, 0.3],
 [7.7, 2.6, 6.9, 2.3]
]

In [ ]:
endpoint = aiplatform.Endpoint('1940778760512995328')
prediction = endpoint.predict(instances=instances)
out = prediction.predictions
out

[1.0, 0.0, 2.0]

In [ ]:
import pandas as pd
data = pd.read_csv('gs://pe-fcor-ec-coea-explore-cs-dev01/MLOps-pipelines/121305041613/ferreycorp-mlops-template-pred-20230810055625/get-data_-8508731260242231296/dataset.csv')
# Define a mapping dictionary for the columns
column_mapping = {
    'sepal length (cm)': 'sepal_length_cm',
    'sepal width (cm)': 'sepal_width_cm',
    'petal length (cm)': 'petal_length_cm',
    'petal width (cm)': 'petal_width_cm',
}

# Rename the columns using the mapping
data.rename(columns=column_mapping, inplace=True)

# Converting the DataFrame to a list of lists
X_eval = data.drop(columns=['target'])
instances = X_eval.values.tolist()

In [ ]:
PROJECT_ID = 'pe-fcor-ec-coea-explore-dev'
REGION = 'us-east4'
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
from pipeline.prod_modules import get_endpoint_by_display_name
endpoint = get_endpoint_by_display_name(display_name = "iris-model-endpoint")

response = endpoint.predict(instances=instances)
predictions = response.predictions

In [ ]:
data = X_eval
data['predictions'] = predictions

In [ ]:
output_bq_project = "pe-fcor-ec-coea-explore-dev"
output_bq_dataset = "Mlops"
output_bq_table = "MlopsTemplate"
credential_path = "key/pe-fcor-ec-coea-explore-dev-dd7a6c961493.json"

import json
# Path to your JSON file
schema_json_path = 'path/to/your/schema.json'
# Read the JSON file
with open(schema_json_path, 'r') as file:
    schema_dict = json.load(file)

from FerreyLib.gcp import bigquery
bigquery.write_df(df              = data, 
                  project_id      = output_bq_project, 
                  dataset_id      = output_bq_dataset, 
                  table_id        = output_bq_table,
                  credential_path = credential_path,
                  if_exists       = 'replace')

In [ ]:
data

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,predictions
0,6.1,2.8,4.7,1.2,1.0
1,5.7,3.8,1.7,0.3,0.0
2,7.7,2.6,6.9,2.3,2.0
3,6.0,2.9,4.5,1.5,1.0
4,6.8,2.8,4.8,1.4,1.0
...,...,...,...,...,...
95,5.5,2.5,4.0,1.3,1.0
96,4.4,2.9,1.4,0.2,0.0
97,4.3,3.0,1.1,0.1,0.0
98,6.0,2.2,5.0,1.5,2.0
